# Directory settings

In [ ]:
import json
with open('../input/train-weights-optimization/best_weights.json', 'r') as f:
    weights_dict = json.load(f)
weights_dict

# Library

In [ ]:
# ====================================================
# Library
# ====================================================
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

import os
import math
import time
import random
import glob
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter
import yaml

import scipy as sp
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

from albumentations import (
    Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightness, RandomContrast, RandomBrightnessContrast, Rotate, ShiftScaleRotate, Cutout, 
    IAAAdditiveGaussianNoise, Transpose, CenterCrop
    )
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import timm

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# CFG

In [ ]:
# # ====================================================
# # CFG
# # ====================================================

normal_configs = []
tta_configs = []
normal_model_dirs = []
tta_model_dirs = []

for model_dir in weights_dict.keys():
    assert len(glob.glob(f'{model_dir}/*.yml'))==1
    config_path = glob.glob(f'{model_dir}/*.yml')[0]
    with open(config_path) as f:
        config = yaml.load(f)
    if 'valid_augmentation' in config['tag'].keys():
        tta_model_dirs.append(model_dir)
        tta_configs.append(config)
    else:
        normal_model_dirs.append(model_dir)
        normal_configs.append(config)


TRAIN_PATH = '../input/cassava-leaf-disease-classification/train_images'
TEST_PATH = '../input/cassava-leaf-disease-classification/test_images'
OUTPUT_DIR = './'

# Utils

In [ ]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    return accuracy_score(y_true, y_pred)


@contextmanager
def timer(name):
    t0 = time.time()
    LOGGER.info(f'[{name}] start')
    yield
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')


# def init_logger(log_file=OUTPUT_DIR+'inference.log'):
#     from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
#     logger = getLogger(__name__)
#     logger.setLevel(INFO)
#     handler1 = StreamHandler()
#     handler1.setFormatter(Formatter("%(message)s"))
#     handler2 = FileHandler(filename=log_file)
#     handler2.setFormatter(Formatter("%(message)s"))
#     logger.addHandler(handler1)
#     logger.addHandler(handler2)
#     return logger

#LOGGER = init_logger()


def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# seed_torch(seed=CFG['seed'])

# Data Loading

In [ ]:
test = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
test.head()

# Dataset

In [ ]:
# ====================================================
# Dataset
# ====================================================
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['image_id'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TEST_PATH}/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image
    
    
class TTADataset(Dataset):
    def __init__(self, df, image_path, ttas):
        self.df = df
        self.file_names = df['image_id'].values
        self.labels = df['label'].values
        self.image_path = image_path
        self.ttas = ttas

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{self.image_path}/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        imglist=[tta(image=image)['image'] for tta in self.ttas]  # update

        image=torch.stack(imglist)
        label = torch.tensor(self.labels[idx]).long()
        
        return image, label

# Transforms

In [ ]:
def _get_augmentations(aug_list, cfg):
    process = []
    for aug in aug_list:
        if aug ==  'Resize':
            process.append(Resize(cfg['size'], cfg['size']))
        elif aug == 'RandomResizedCrop':
            process.append(RandomResizedCrop(cfg['size'], cfg['size']))
        elif aug == 'CenterCrop':
            process.append(CenterCrop(CFG['size'], CFG['size']))
        elif aug == 'Transpose':
            process.append(Transpose(p=0.5))
        elif aug == 'HorizontalFlip':
            process.append(HorizontalFlip(p=0.5))
        elif aug == 'VerticalFlip':
            process.append(VerticalFlip(p=0.5))
        elif aug == 'ShiftScaleRotate':
            process.append(ShiftScaleRotate(p=0.5))
        elif aug == 'Normalize':
            process.append(Normalize(
                            mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225],
                        ))
        else:
            raise ValueError(f'{aug} is not suitable')

    process.append(ToTensorV2())

    return process

In [ ]:
# ====================================================
# Transforms
# ====================================================
def get_transforms(*, aug_list, cfg):
    
    return Compose(
        _get_augmentations(aug_list, cfg)
    )

In [ ]:
def get_ttas(cfg):
    norm_mean = [0.485, 0.456, 0.406]
    norm_std = [0.229, 0.224, 0.225]

    oneof_augs = [
        CenterCrop(cfg['size'], cfg['size']), 
        Resize(cfg['size'], cfg['size'])
    ]

    ttas = [[
        Compose([
            oneof_aug,
            Normalize(mean=norm_mean, std=norm_std, p=1.),
            ToTensorV2()
        ]),
        Compose([
            oneof_aug,
            Transpose(p=1),
            Normalize(mean=norm_mean, std=norm_std, p=1.),
            ToTensorV2()
        ])
    ] for oneof_aug in oneof_augs]

    # 平滑化
    ttas = sum(ttas, [])
    
    return ttas

# MODEL

In [ ]:
# ====================================================
# MODEL
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, model_name, target_size, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        if hasattr(self.model, 'classifier'):
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(n_features, target_size)
        elif hasattr(self.model, 'fc'):
            n_features = self.model.fc.in_features
            self.model.fc = nn.Linear(n_features, target_size)
        elif hasattr(self.model, 'head'):
            n_features = self.model.head.in_features
            self.model.head = nn.Linear(n_features, target_size)

    def forward(self, x):
        x = self.model(x)
        return x

# Helper functions

In [ ]:
# ====================================================
# Helper functions
# ====================================================
def inference_normal(model, states, test_loader, device):
    model.to(device)
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    probs = []
    for i, (images) in tk0:
        images = images.to(device)
        avg_preds = []
        for state in states:
            model.load_state_dict(state['model'])
            model.eval()
            with torch.no_grad():
                y_preds = model(images)
            avg_preds.append(y_preds.softmax(1).to('cpu').numpy())
        avg_preds = np.mean(avg_preds, axis=0)
        probs.append(avg_preds)
    probs = np.concatenate(probs)
    return probs


def inference_tta(model, states, tta_loader, device):
    model.to(device)
    tk0 = tqdm(enumerate(tta_loader), total=len(tta_loader))
    probs = []
    for i, (images, _) in tk0:
        images = images.to(device)
        batch_size, n_crops, c, h, w = images.size()
        images = images.view(-1, c, h, w)
        
        avg_preds = []
        for state in states:
            model.load_state_dict(state['model'])
            model.eval()
            with torch.no_grad():
                y_preds = model(images).softmax(1)
                y_preds = y_preds.view(batch_size, n_crops,-1)
            avg_preds.append(y_preds.to('cpu').numpy())
        avg_preds = np.mean(avg_preds, axis=0)
        probs.append(avg_preds)
        del images, _, y_preds, avg_preds
        torch.cuda.empty_cache()
    probs = np.concatenate(probs)
    return probs.mean(1)

# inference

In [ ]:
def main(config, model_dir):
    # ====================================================
    # inference
    # ====================================================
    
    INFO = config['info']
    TAG = config['tag']
    CFG = config['cfg']
    CFG['train'] = False
    CFG['inference'] = True
    inference_batch_size = 64
    
    seed_torch(seed=CFG['seed'])

    model = CustomModel(TAG['model_name'], CFG['target_size'], pretrained=False)
    states = [torch.load(path) for path in glob.glob(f'{model_dir}/*.pth')]
    test_dataset = TestDataset(test, transform=get_transforms(aug_list=['Resize', 'Normalize'], cfg=CFG))
    test_loader = DataLoader(test_dataset, batch_size=inference_batch_size, shuffle=False, 
                             num_workers=CFG['num_workers'], pin_memory=True)
    predictions = inference_normal(model, states, test_loader, device)
    
    return predictions



def main_tta(config, model_dir):
    # ====================================================
    # inference
    # ====================================================
    
    INFO = config['info']
    TAG = config['tag']
    CFG = config['cfg']
    CFG['train'] = False
    CFG['inference'] = True
    inference_batch_size = 8
    
    seed_torch(seed=CFG['seed'])

    model = CustomModel(TAG['model_name'], CFG['target_size'], pretrained=False)
    states = [torch.load(path) for path in glob.glob(f'{model_dir}/*.pth')]
    ttas = get_ttas(CFG)
    tta_dataset = TTADataset(test, TEST_PATH, ttas=ttas)
    tta_loader = DataLoader(tta_dataset, batch_size=inference_batch_size, shuffle=False, 
                             num_workers=2, pin_memory=True)
    predictions = inference_tta(model, states, tta_loader, device)
    
    return predictions

In [ ]:
# ensemble
predictions_list = []
model_dir_list = []
for config, model_dir in zip(normal_configs, normal_model_dirs):
    predictions_list.append(main(config, model_dir))
    model_dir_list.append(model_dir)
    
for config, model_dir in zip(tta_configs, tta_model_dirs):
    predictions_list.append(main_tta(config, model_dir))
    model_dir_list.append(model_dir)


# predictions = (predictions_list[0] + predictions_list[1]) / 2
# # submission
# test['label'] = predictions.argmax(1)
# test[['image_id', 'label']].to_csv(OUTPUT_DIR+'submission.csv', index=False)
# test.head()

# submit

In [ ]:
# ensemble
predictions = np.zeros(predictions_list[0].shape, dtype=predictions_list[0].dtype)
for i, key in zip(range(len(predictions_list)), model_dir_list):
    predictions += predictions_list[i] * weights_dict[key]
# submission
test['label'] = predictions.argmax(1)
test[['image_id', 'label']].to_csv(OUTPUT_DIR+'submission.csv', index=False)
test.head()